## Import libraries

In [ ]:
%cd ~/bdp_bipm

In [ ]:
import os, json, sys
import shutil
import multiprocessing
import copy
from glob import glob
import urllib
import pickle
import urllib.request
from imp import reload
from PIL import Image

import pandas as pd
import numpy as np
import random
from numpy import zeros
from scipy.linalg import svd

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.lancaster import LancasterStemmer
from nltk.tag.stanford import StanfordPOSTagger
from nltk.corpus import words as nltk_words

from scipy import misc
from textblob import TextBlob
import textacy
import spacy
from fuzzywuzzy import fuzz

import gensim, logging
from gensim import similarities
from gensim.models import Phrases, word2vec, doc2vec
from gensim.models.phrases import Phraser
from gensim.models.keyedvectors import KeyedVectors
import time
from functools import reduce
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
import xgboost as xgb

import seaborn as sns
import string
import math

from wordcloud import WordCloud
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import linear_model

from keras.layers import Activation
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [ ]:
%matplotlib inline
%cd ~/bdp_bipm/data
!ls

# Import Data From MongoDB

In [ ]:

import scrapy

class BlogSpider(scrapy.Spider):
    name = 'blogspider'
    start_urls = ['https://blog.scrapinghub.com']

    def parse(self, response):
        for title in response.css('h2.entry-title'):
            yield {'title': title.css('a ::text').extract_first()}

        for next_page in response.css('div.prev-post > a'):
            yield response.follow(next_page, self.parse)

try:
    # Python 3.x
    from urllib.parse import quote_plus
except ImportError:
    # Python 2.x
    from urllib import quote_plus
#Connection to SERVER

#Local connection
uri = "mongodb://localhost:27017"

client = MongoClient(uri)

db = client.get_database("test")
assert db.name == 'test'

class CNames:
    id = "_id"
    auto_category_id = "auto_category_id"
    foreign_category_label = "foreign_category_label"
    source = "source"
    traderId = "traderId"
    is_from_marketplace = "is_from_marketplace"
    url = "url"
    type = "type"
    start_bid = "start_bid"
    min_bid = "min_bid"
    original_images = "original_images"
    processed_images = "processed_images"
    manufacturer_normalized = "manufacturer_normalized"
    manufacturer = "manufacturer"
    machine_type = "machine_type"
    location = "location"
    lang = "lang"
    currency = "currency"
    country = "country"
    city = "city"
    created_at = "created_at"
    categoriesKeys = "categoriesKeys"
    bid_count = "bid_count"
    actual_bid = "actual_bid"
    year_of_manufacture = "year_of_manufacture"
    attributes = "attributes"
    auctions = "auctions"
    key = "key"
    positionlat = "position.lat"
    positionlon = "position.lon"
    tags = "tags"
    target = "auto_category_id"

#--------------------------------------------------------------------------------------------------------------------------------
experimentSize = 1000000



lotsAttribListToFindInDb = {"_id":False,"auto_category_id":True,"foreign_category_label":True,"source":True,"traderId":True,"is_from_marketplace":True,
                            "url":True,"type":True,"translations":True,"start_bid":True,"min_bid":True,"original_images":True,"processed_images":True,"manufacturer_normalized":True,
                            "manufacturer":True,"machine_type":True,"location":True,"lang":True,"currency":True,"country":True,"city":True,"created_at":True,
                            "categoriesKeys":True,"bid_count":True,"actual_bid":True,"year_of_manufacture":True,"attributes":True,"position":True}

catsAttribListToFindInDb = {"_id":True,"auctions":True,"created_at":True,"key":True,"translations":True,"position":True,"tags":True,"parent_id":True}
#sample_object = {'Name':'John', 'Location':{'City':'Los Angeles','State':'CA'}}
#json_normalize(sample_object)


lotsCursor = db.auctions.find({},lotsAttribListToFindInDb)
catsCursor = db.catjsontest.find({},catsAttribListToFindInDb,limit = experimentSize)

# Convert to DF

#print(type(catsCursor[1:100]))
#cats100_df = json_normalize(catsCursor[4])
#replace print(i) with single json creation for DF
def dfFromCursor(cur):
    rawList = []
    for i in cur:
        rawList += [i]

    return pd.DataFrame(json_normalize(rawList))

# dtype={ColumnNames.actual_bid: np.uint32,
#                                                        ColumnNames.bid_count: np.uint16,
#                                                        ColumnNames.year_of_manufacture: np.uint16,
#                                                        ColumnNames.start_bid: np.uint32,
#                                                        ColumnNames.is_from_marketplace: np.bool_,
#                                                        ColumnNames.positionlat: np.float32,
#                                                        ColumnNames.positionlon: np.float32
#                                                        }
df_categories = dfFromCursor(catsCursor)
df_lots = dfFromCursor(lotsCursor)

df_lots.to_pickle("df_lots_all.pkl")
df_categories.to_pickle("df_categories.pkl")
#DONE
df_lots = pd.read_pickle("df_lots1m.pkl")
df_categories = pd.read_pickle("df_categories.pkl")

## Define constants

In [ ]:
pickle_dir = "pickles/"

#stTagger = StanfordPOSTagger(model_filename='../stanford-postagger-full-2016-10-31/models/english-bidirectional-distsim.tagger', 
#                       path_to_jar= '../stanford-postagger-full-2016-10-31/stanford-postagger.jar')
w2vModel = KeyedVectors.load_word2vec_format('glove_6B_50d.txt', binary=False)
lancas = LancasterStemmer()
wnl = nltk.WordNetLemmatizer()

prepositions = set(["by","around", "at", "away", "down", "in", "off", "on", "out", "over", "round", "up", "upon", "onto", "through", "against", "under", "aside", "between", "before", "with", "without", "forward", "behind", "without", "across", "as", "back", "apart", "after", "away", "towards", "ahead", "along", "of", "past", "aback", "about", "above", "apart", "aside", "way", "to"])
negationWords = set(["not","without","no","never"])
POS_list = ["$","''","(",")",",","--",".",":","CC","CD","DT","EX","FW","IN","JJ","JJR","JJS","LS","MD","NN","NNP","NNPS","NNS","PDT","POS","PRP","PRP$","RB","RBR","RBS","RP","SYM","TO","UH","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP","WP$","WRB","``"]
NE_list = set(["ORGANIZATION","PERSON","LOCATION","DATE","TIME","MONEY","PERCENT","FACILITY","GPE"])
posToWn = {"N":wn.NOUN,"V":wn.VERB,"J":wn.ADJ,"R":wn.ADV}

all_columns_set = set(["actual_bid",
"attributes",
"auto_category_id",
"bid_count",
"categoriesKeys",
"city",
"country",
"created_at",
"currency",
"foreign_category_label",
"is_from_marketplace",
"lang",
"location",
"machine_type",
"manufacturer",
"manufacturer_normalized",
"min_bid",
"original_images",
"position",
"position.lat",
"position.lon",
"processed_images",
"source",
"start_bid",
"traderId",
"translations.ar.description",
"translations.ar.title",
"translations.bg.description",
"translations.bg.title",
"translations.cs.description",
"translations.cs.title",
"translations.da.description",
"translations.da.title",
"translations.de.description",
"translations.de.original_title",
"translations.de.slug",
"translations.de.title",
"translations.en.description",
"translations.en.original_title",
"translations.en.slug",
"translations.en.title",
"translations.es.description",
"translations.es.original_title",
"translations.es.title",
"translations.et.description",
"translations.et.title",
"translations.fi.description",
"translations.fi.title",
"translations.fr.description",
"translations.fr.original_title",
"translations.fr.title",
"translations.hr.description",
"translations.hr.title",
"translations.hu.description",
"translations.hu.title",
"translations.it.description",
"translations.it.title",
"translations.ko.description",
"translations.ko.title",
"translations.nl.description",
"translations.nl.title",
"translations.no.description",
"translations.no.title",
"translations.pl.description",
"translations.pl.title",
"translations.ro.description",
"translations.ro.title",
"translations.ru.description",
"translations.ru.title",
"translations.sk.description",
"translations.sk.title",
"translations.sl.description",
"translations.sl.title",
"translations.sv.description",
"translations.sv.title",
"translations.tr.description",
"translations.tr.title",
"type",
"url"])

numeric_columns_list = ["year_of_manufacture",
"bid_count",
"start_bid",
"actual_bid",
"min_bid",
"position.lat",
"position.lon",
"is_from_marketplace"]

translate_columns = [
    "translations.ar.description",
"translations.ar.title",
"translations.bg.description",
"translations.bg.title",
"translations.cs.description",
"translations.cs.title",
"translations.da.description",
"translations.da.title",
"translations.de.description",
"translations.de.original_title",
"translations.de.slug",
"translations.de.title",
"translations.en.description",
"translations.en.original_title",
"translations.en.slug",
"translations.en.title",
"translations.es.description",
"translations.es.original_title",
"translations.es.title",
"translations.et.description",
"translations.et.title",
"translations.fi.description",
"translations.fi.title",
"translations.fr.description",
"translations.fr.original_title",
"translations.fr.title",
"translations.hr.description",
"translations.hr.title",
"translations.hu.description",
"translations.hu.title",
"translations.it.description",
"translations.it.title",
"translations.ko.description",
"translations.ko.title",
"translations.nl.description",
"translations.nl.title",
"translations.no.description",
"translations.no.title",
"translations.pl.description",
"translations.pl.title",
"translations.ro.description",
"translations.ro.title",
"translations.ru.description",
"translations.ru.title",
"translations.sk.description",
"translations.sk.title",
"translations.sl.description",
"translations.sl.title",
"translations.sv.description",
"translations.sv.title",
"translations.tr.description",
"translations.tr.title"
]

analysed_languages = ["de","en"]

translation_attr = ["description","title"]


target_column = "auto_category_id"
sample_of_rows_size = 100000 #
sample_of_cats_size = 100 # 
#Init spacy nlp
en_nlp = spacy.load('en')
de_nlp = spacy.load('de')

## Tuning parameters

In [ ]:
class TuningParameters:
    title_importance = 1.0
    form_phrases_count = 10
    phrases_bigram_weight = 1.0
    phrases_trigram_weight = 1.0
    titles_weight = 1.0
    fuzzy_word_threshold = 77
    
number_of_cores = max(multiprocessing.cpu_count() - 5,1)

## Parallelism functions

In [ ]:
mydata = [{'subid' : 'B14-111', 'age': 75, 'fdg':1.78},
          {'subid' : 'B14-112', 'age': 22, 'fdg':1.56},
            {'subid' : 'B14-113', 'age': 24, 'fdg':1.59},
         {'subid' : 'B14-111', 'age': 78, 'fdg':1.88},
          {'subid' : 'B14-111', 'age': 32, 'fdg':1.06},
            {'subid' : 'B14-113', 'age': 34, 'fdg':1.19},
         {'subid' : 'B14-111', 'age': 75, 'fdg':1.78},
          {'subid' : 'B14-112', 'age': 22, 'fdg':1.56},
            {'subid' : 'B14-113', 'age': 24, 'fdg':1.59},
         {'subid' : 'B14-111', 'age': 78, 'fdg':1.88},
          {'subid' : 'B14-112', 'age': 32, 'fdg':1.06},
            {'subid' : 'B14-113', 'age': 34, 'fdg':1.19}]
df = pd.DataFrame(mydata)

def _apply_df(args):
    df, func, kwargs = args
    return func(df,**kwargs) 

def apply_by_multiprocessing(df, func,  **kwargs):
    workers = kwargs.pop('workers')
    repeat = kwargs.pop('repeat')
    
    for i in range(repeat):
        pool = multiprocessing.Pool(processes=workers)
        result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers<<1)])
        pool.close()
        #pool.join()
        df = pd.concat(result)
    
    return df

def apply_by_multiprocessing_corpus(df, func,  **kwargs):
    workers = kwargs.pop('workers')
    
    pool = multiprocessing.Pool(processes=workers)
    n = workers<<1
    result = pool.map(_apply_df, [(d, func, kwargs)
        for d in [df[i:i + n] for i in range(0, len(df), n)]])
    
    pool.close()
    return sum(result)


def group_by_target_column(dff):
    return dff.groupby("subid").agg(sum)

def sqrr(x):
    return sum(x)

print(group_by_target_column(apply_by_multiprocessing(df,group_by_target_column,repeat=2,workers = 3)))
apply_by_multiprocessing_corpus([1,2,3,4,5,6,7,8,9,10],sqrr,workers=4)

### Read input

In [ ]:
df_lots = pd.read_pickle(pickle_dir+"df_lots_all.pkl")
df_categories = pd.read_pickle(pickle_dir+"df_categories.pkl")

df_lots = df_lots[df_lots[target_column].notnull()]
def boolToInt(x):
    if x == "True":
        return 1
    else:
        return 0
df_lots["is_from_marketplace"] = df_lots["is_from_marketplace"].apply(boolToInt)

# Preprocessing Data for exploration and further use

#### Init. of list of columns for text mining 

In [ ]:
#Create dataframe by concatenated titles and descriptions grouped by categories
grouping_non_translate_columns = [
"foreign_category_label",
"machine_type",
"manufacturer",
"manufacturer_normalized",target_column]+["translations.%s.%s" % (lang, trans) for lang in analysed_languages for trans in translation_attr]

#### Slice text mining columns from DF 

In [ ]:
grouping_titles_descriptions = df_lots[grouping_non_translate_columns].sample(sample_of_rows_size)

In [ ]:
sample_df_lots = df_lots.sample(sample_of_rows_size)

In [ ]:
sample_df_lots = sample_df_lots[grouping_non_translate_columns]

#### Util. functions for grouping dataframe by target column

In [ ]:
def en_text_nlp(st):
    #print("ser ",en_nlp(str(series)))
    if type(st) == str:
        #print("st = " ,en_nlp(st.rstrip()))
        return en_nlp(str(st))
    else:
        return en_nlp("")
    

def de_text_nlp(st):
    #print("ser ",en_nlp(str(series)))
    if type(st) == str:
        #print("st = " ,en_nlp(st.rstrip()))
        return de_nlp(str(st))
    else:
        return de_nlp("")

def series_nlp(series):
    if series.name != target_column:
        if ".de." not in series.name:
            return series.where(series.isnull(),en_text_nlp)
        else:
            return series.where(series.isnull(),de_text_nlp)
    else:
        return series

def text_sum(series):
    if series[series.notnull()].size > 1:
        return reduce(lambda x, y: str(x) +" " + str(y), series[series.notnull()])
    else:
        return ""
    
def text_sum_nlp(series):
    if series[series.notnull()].size > 1:
        txt = reduce(lambda x, y: str(x) +" " + str(y), series[series.notnull()])
        if ".de." not in series.name:
            #print(txt)
            return en_text_nlp(txt)
        else:
            return de_text_nlp(txt)
    else:
        return ""

def df_text_grouped_nlp(dff,**kwargs):
    return dff.groupby(target_column).agg(text_sum_nlp)


def par_doc_series(series,lang_nlp):
    return pd.Series(np.array([doc for doc in lang_nlp.pipe(series.values, batch_size=400,n_threads=number_of_cores)]))
        
def df_text_nlp(dff):
    #dff.drop(target_column,axis=1)[["translations.en.title","translations.en.description"]]
    d = {
     "translations.en.title":par_doc_series(dff["translations.en.title"].astype(str),en_nlp),
     "translations.en.description":par_doc_series(dff["translations.en.description"].astype(str),en_nlp),
     "translations.de.title":par_doc_series(dff["translations.de.title"].astype(str),de_nlp),
     "translations.de.description":par_doc_series(dff["translations.de.description"].astype(str),de_nlp)
    }
    resDf = dff.loc[:,[target_column,"foreign_category_label","machine_type","manufacturer","manufacturer_normalized"]]
    for k,v in d.items():
        resDf[k] = v
    
    return resDf

def group_by_target_column_agg(dff):
    return df_text_nlp(dff.groupby(target_column).agg(text_sum).copy())

def isBothGtThan0(st1,st2):
    return len(st1)>0 and len(st2)>0

def fuzzyMatch(st1,st2):
    """
    Just fuzzy partial_token_sort_ratio(https://pypi.python.org/pypi/fuzzywuzzy), which returns the distance between two words
    """
    if not isBothGtThan0(st1,st2):
        return None
    
    return fuzz.partial_token_sort_ratio(st1,st2)

### Failed experiments

In [ ]:
# starttime = time.time()   
# if __name__ == '__main__':
#     #grouped_titles_descriptions = apply_by_multiprocessing(grouping_titles_descriptions[:10000], group_by_target_column_agg, repeat=1, axis=1,workers = number_of_cores)
#     grouped_titles_descriptions = group_by_target_column_agg(grouping_titles_descriptions[:100000])
#     print("Grouped done!")
    
#     #ungrouped_titles_descriptions = df_text_nlp(grouping_titles_descriptions[:100])
    
#     ungrouped_titles_descriptions = df_text_nlp(grouping_titles_descriptions[:100000])
#     print("Ungrouped done!")
# print("Operation completed in ",time.time()-starttime," seconds")
#Full require Operation completed in  11929.308161735535  seconds

In [ ]:
#starttime = time.time()    
#print("Pickling started!")
#grouped_titles_descriptions.to_pickle(pickle_dir+"grouped_by_cat_text_columns.pkl")
#ungrouped_titles_descriptions.to_pickle(pickle_dir+"ungrouped_by_cat_text_columns.pkl")
#grouped_titles_descriptions.info()
#print("Operation completed in ",time.time()-starttime," seconds")

In [ ]:
#spacy doc and pickle is uncompactable 
#grouped_titles_descriptions = pd.read_pickle(pickle_dir+"grouped_by_cat_text_columns_all.pkl")
#ungrouped_titles_descriptions = pd.read_pickle(pickle_dir+"ungrouped_by_cat_text_columns_all.pkl")

In [ ]:
#print(grouped_titles_descriptions.shape)
#print(ungrouped_titles_descriptions.shape)

## Corpuses and DocTermMatrixes

#### Group DF by category by text mining columns

In [ ]:
#grouped_titles_descriptions = grouping_titles_descriptions.groupby(target_column).agg(text_sum)

In [ ]:
#grouped_titles_descriptions.to_pickle(pickle_dir+"grouped_by_cat_text_columns.pkl")

#### Draft

In [ ]:
#grouped_titles_descriptions.loc[:,["translations.%s.%s" % (lang, "description") for lang in analysed_languages]] = [["12","13"]]
#grouped_titles_descriptions.loc["531600ae78f860d0398b4567","translations.en.description"]=["dasfaf","fsaf f@%%"]

In [ ]:
set_of_cols = list(set(grouping_non_translate_columns).difference(set([target_column])))

In [ ]:
grouped_titles_descriptions= pd.read_pickle(pickle_dir+"grouped_by_cat_text_columns.pkl")

def flatten1Level(ser):
    norm_grouped_titles_descriptions = []
    for i in ser.values.tolist():
        if type(i) == list:
            if len(i)>1:
                j = reduce(lambda x, y: x +" " + y, i)                
                norm_grouped_titles_descriptions += [j]
            else:
                norm_grouped_titles_descriptions += i
        else:
            norm_grouped_titles_descriptions += [i]
        
    return pd.Series(np.array(norm_grouped_titles_descriptions),index = ser.index)
#Flatten descriptions


Combine all columns to get "full" description
grouped_titles_descriptions[["translations.%s.%s" % (lang, "description") for lang in analysed_languages]] = grouped_titles_descriptions[["translations.%s.%s" % (lang, "description") for lang in analysed_languages]].apply(flatten1Level, axis=0)

####  Rename indexes of Series (draft)

In [ ]:
#for lang in analysed_languages:
#    grouped_titles_descriptions["translations.%s.description" % lang] = test["translations.%s.description" % lang].rename({i:grouped_titles_descriptions["translations.%s.description" % lang].index[i] for i in range(869)})

In [ ]:
set_of_cols_en = list(set(set_of_cols).difference(set(["translations.de.title","translations.de.description"])))
set_of_cols_de = list(set(set_of_cols).difference(set(["translations.en.title","translations.en.description"])))

In [ ]:
def convertToEnDocSeries(x):
    return x.apply(textacy.Doc,args=(None,"en"))

def convertToEnDoc(x):
    return textacy.Doc(x,lang="en")

In [ ]:
sample_df_lots["transtions.en.full.description"] = sample_df_lots.loc[:,set_of_cols_en].apply(text_sum,reduce = True, axis = 1)

In [ ]:
sample_df_lots.set_index(target_column,inplace=True)

In [ ]:
sample_df_lots["transtions.en.full.description"].shape

In [ ]:
#sample_df_lots["transtions.en.full.description_doc"] = sample_df_lots.loc[:,"transtions.en.full.description"].apply(convertToEnDoc)

In [ ]:
grouped_titles_descriptions["transtions.en.full.description"] = grouped_titles_descriptions[set_of_cols_en].apply(text_sum,reduce = True, axis = 1)
grouped_titles_descriptions["transtions.de.full.description"] = grouped_titles_descriptions[set_of_cols_de].apply(text_sum,reduce = True, axis = 1)

In [ ]:
#grouped_titles_descriptions.to_pickle(pickle_dir+"grouped_by_cat_text_columns_full.pkl")
grouped_titles_descriptions = pd.read_pickle(pickle_dir+"grouped_by_cat_text_columns_full.pkl").sample(sample_of_cats_size)

In [ ]:
grouped_titles_descriptions

In [ ]:
LOGGER = logging.getLogger(__name__)

_DEFAULT_N_THREADS = max(multiprocessing.cpu_count() - 1, 1)
class ParCorpus(textacy.Corpus):
    def add_texts(self, texts, metadatas=None,
                  n_threads=_DEFAULT_N_THREADS, batch_size=1000):
        super(ParCorpus, self).add_texts(texts, metadatas=metadatas,n_threads=n_threads, batch_size=5)

In [ ]:
# buf = open("error.json","w")
# test_str = records_en[5]['value'].text
# buf.write(test_str)
# buf.close()#21904

### Generate text stream

In [ ]:
def seriesToDict(ser):
    return [{"key":key,"value":value} for key, value in zip(ser.index,ser)]

def streamAndMeta(ser):
    return textacy.fileio.split_record_fields(seriesToDict(ser), 'value')

def corpusFromSeriesDoc(ser,lang="en"):
    text_stream_en, metadata_stream_en = streamAndMeta(ser)
    return textacy.Corpus(lang, docs=text_stream_en, metadatas = metadata_stream_en)

def corpusFromSeriesText(ser,lang="en"):
    text_stream_en, metadata_stream_en = streamAndMeta(ser)
    return textacy.Corpus(lang, texts=text_stream_en, metadatas = metadata_stream_en)

In [ ]:
starttime = time.time()
# Check if the word exist "word" in words.words()
#open file example
#for subdir, dirs, files in os.walk(vendors_dir):
#    for file in files:
#        f = open(os.path.join(subdir, file), errors='ignore')
#        erpDocs[file] = textacy.preprocess.fix_bad_unicode(textacy.preprocess.normalize_whitespace(f.read()))
#records_en = [{"key":key,"value":"# "+value.encode('utf8').decode('utf8','replace')} for key, value in grouped_titles_descriptions["transtions.en.full.description"].to_dict().items()]
records_en = [{"key":key,"value":"# "+value} for key, value in grouped_titles_descriptions["transtions.en.full.description"].to_dict().items()]
records_de = [{"key":key,"value":"# "+value} for key, value in grouped_titles_descriptions["transtions.de.full.description"].to_dict().items()]

print("Finish time ",time.time()-starttime)

## Phraser

In [ ]:

phrases_en = Phrases([f['value'].split(" ") for f in records_en], min_count=TuningParameters.form_phrases_count, threshold=7)

bigramPhraserEn = Phraser(phrases_en)

phrases_de = Phrases([f['value'].split(" ") for f in records_de], min_count=TuningParameters.form_phrases_count, threshold=7)

bigramPhraserDe = Phraser(phrases_de)

# sentence_stream_phrases = bigramPhraserEn[sentence_stream]

# trigram = Phrases(bigramPhraserEn[sentence_stream])

# sent = [u'the', u'new', u'york', u'times', u'is', u'a', u'newspaper']
# print(trigram[bigram[sent]])

In [ ]:
records_de_c = copy.deepcopy(records_de)
records_en_c = copy.deepcopy(records_en)

for f in records_en_c:
    f['value'] = " ".join(bigramPhraserEn[f['value'].split(" ")])

for f in records_de_c:
    f['value'] = " ".join(bigramPhraserDe[f['value'].split(" ")])

In [ ]:
starttime = time.time()
text_stream_de, metadata_stream_de = textacy.fileio.split_record_fields(records_de, 'value')
text_stream_en, metadata_stream_en = textacy.fileio.split_record_fields(records_en, 'value')
corpuses = {}
corpuses['normal.en'] = ParCorpus("en", texts=text_stream_en, metadatas = metadata_stream_en)
corpuses['normal.de'] = ParCorpus('de', texts=text_stream_de, metadatas = metadata_stream_de)

text_stream_de, metadata_stream_de = textacy.fileio.split_record_fields(records_de_c, 'value')
text_stream_en, metadata_stream_en = textacy.fileio.split_record_fields(records_en_c, 'value')
corpuses['phrases.en'] = ParCorpus("en", texts=text_stream_en, metadatas = metadata_stream_en)
corpuses['phrases.de'] = ParCorpus('de', texts=text_stream_de, metadatas = metadata_stream_de)
print("Finish time ",time.time()-starttime)
#for 35 Finish time  1040.6533074378967

In [ ]:
lot_corpuses = {}
lot_corpuses['normal.en.10k'] = corpusFromSeriesText(sample_df_lots["transtions.en.full.description"])

In [ ]:
lot_corpuses

In [ ]:
lot_corpuses['normal.en'][1].metadata

In [ ]:
sample_df_lots.shape

## Noun chunks

In [ ]:
starttime = time.time()

def plusWithSpace(x,y):
    return x+" "+y

def reduceWithUnderScore(l):
    reduce(plusWithSpace,l)
res = []    

def callback(result):
    res.append(result)
    

def listoflistTOlistofstrLabel(lol):
    pool = multiprocessing.Pool(processes=number_of_cores)
    
    for doc in lol:
        pool.apply_async(reduceWithUnderScore, args = ([token.text.replace(" ","_") for token in doc],),callback=callback)

    pool.close()
    pool.join()
    return res
    #return [reduce(lambda x,y:x+" "+y,[token.text.replace(" ","_") for token in doc]) for doc in lol]

sentence_stream_noun = listoflistTOlistofstrLabel([[w for w in textacy.extract.noun_chunks(doc)] for doc in corpuses['normal.en'][:2]])
#pattern = textacy.constants.POS_REGEX_PATTERNS['en']['NP']
print("Finish time ",time.time()-starttime)

In [ ]:
#Test of chunk
#len(list(textacy.extract.pos_regex_matches(doc, pattern)))

In [ ]:
corpuses["chunk.en"] = textacy.Corpus('en', texts=sentence_stream_noun)

In [ ]:
#TOO Time consuming
#corpuses["chunk.de"] = textacy.Corpus('de', texts=sentence_stream_noun)

In [ ]:
corpuses['normal.de'].save(pickle_dir+"docs", name='Normal_Grouped_Desc_Title_DE_100', compression='gzip')
corpuses['normal.en'].save(pickle_dir+"docs", name='Normal_Grouped_Desc_Title_EN_100', compression='gzip')

In [ ]:
# corpuses['normal.de'].save(pickle_dir+"docs", name='Normal_Grouped_Desc_Title_DE', compression='gzip')
# corpuses['normal.en'].save(pickle_dir+"docs", name='Normal_Grouped_Desc_Title_EN', compression='gzip')

In [ ]:
corpuses['normal.de'] = textacy.Corpus.load(pickle_dir+"docs", name='Normal_Grouped_Desc_Title_DE', compression='gzip')
corpuses['normal.en'] = textacy.Corpus.load(pickle_dir+"docs", name="Normal_Grouped_Desc_Title_EN", compression='gzip')

In [ ]:
starttime = time.time()
corpuses['normal.de'] = textacy.Corpus.load(pickle_dir+"docs", name='Normal_Grouped_Desc_Title_DE_35', compression='gzip')
corpuses['normal.en'] = textacy.Corpus.load(pickle_dir+"docs", name="Normal_Grouped_Desc_Title_EN_35", compression='gzip')
print("Success! in ",time.time()-starttime)

### Alternative pickling 

In [ ]:
# for i,doc in enumerate(corpuses['normal.en']):

#     try:
#         doc.save(pickle_dir+"docs/en",name="doc"+str(i))
#     except KeyError:
        
#         print(doc," number "+str(i))

In [ ]:
# for i,doc in enumerate(corpuses['normal.en']):
#     #corpuses['normal.de'].save(pickle_dir+"docs", name='Normal_Grouped_Desc_Title_DE', compression='gzip')
#     #corpuses['normal.en'].save(pickle_dir+"docs", name='Normal_Grouped_Desc_Title_EN')
#     try:
#         doc.save(pickle_dir+"docs/de",name="doc"+str(i))
#     except KeyError:
#         print(doc," number "+str(i))

### Error line checking

In [ ]:
# exDoc = en_nlp(corpuses['normal.en'][31].text)

In [ ]:
#[[token.orth_ for token in textacy.extract.words(doc, filter_stops=True, filter_punct=True, filter_nums=True)] for doc in corpuses['normal'] ]

## Not words list just for English, for German doesn't make a sense

In [ ]:
nltk_dictionary = dict.fromkeys(nltk_words.words(), None)

def is_english_word(word):
    try:
        x = dictionary[word]
        return True
    except KeyError:
        return False

## Add category names to dataframe

In [ ]:
df_categories

# Data exploration

In [ ]:
#Data exploration
df_lots.shape

In [ ]:
corpuses['normal.de']

In [ ]:
corpuses['normal.en']

In [ ]:
def createStats(corp):
    stats_array = []
    for doc in corp:
        if len(doc)>1:
            ts = textacy.text_stats.TextStats(doc)
            d = ts.basic_counts
            d["readability"] = ts.flesch_kincaid_grade_level
            stats_array += [d]
    return stats_array

In [ ]:
stats_list = createStats(corpuses['normal.en'])

In [ ]:
stats_df = pd.DataFrame(stats_list)

In [ ]:
stats_df = pd.read_csv("~/stats_for_100_random_categories.csv")

In [ ]:
stats_df.describe()

In [ ]:


scaler = StandardScaler()

#pipeline = Pipeline([
#    ('vect', CountVectorizer()),
#    ('tfidf', TfidfTransformer()),
#    ('clf', SGDClassifier()),
#])
stats_df = pd.DataFrame(scaler.fit_transform(stats_df[stats_df<400000]), columns=stats_df.columns)
pltt = sns.boxplot(stats_df)
plt.xticks(rotation=45)
#pltt.save

In [ ]:
nullsSeries = np.sum(df_lots.isnull())
#Nulls distribution
sns.boxplot(nullsSeries)

In [ ]:
!pwd

In [ ]:
nonNumericNonTranslationColumns = all_columns_set.difference(set(numeric_columns_list)).difference(set(translate_columns))
# f, subplotax = plt.subplots(4,5)
# f.tight_layout()
# f

ind = np.arange(20)
width = 0.35
fig = plt.figure()
plt.bar(ind, [np.sum(df_lots[col].isnull()) for col in nonNumericNonTranslationColumns], width, color='r')
plt.xticks(ind, nonNumericNonTranslationColumns, rotation='vertical')
fig.savefig("nonTranslationNulls.png")

In [ ]:
df_lots.describe()

In [ ]:
df_lots.info()

In [ ]:
#Explore format of attributes
df_lots["attributes"].iloc[999970][0]["language"]

In [ ]:
#How many images do we have
all_images_by_category = pd.read_pickle(pickle_dir+"all_images_by_category.pkl")

#image_urls = all_images_list[:sample_sample_size]

#df_lots[df_lots["foreign_category_label"].notnull()].apply( )
#Plot title and desc word distribution

In [ ]:
all_images_by_category["amount"] = all_images_by_category["processed_images"].apply(len)

In [ ]:
fig2 = sns.boxplot("amount",data=all_images_by_category[all_images_by_category["amount"]<500])
fig2.get_figure().savefig("numberofimagespercategory.png")

In [ ]:
all_images_by_category.to_csv("~/images")

In [ ]:
en_desc_length = df_lots[df_lots["translations.en.description"].notnull()].loc[:10000,"translations.en.description"].apply(lambda x:len(en_nlp(x)))
en_title_length = df_lots[df_lots["translations.en.title"].notnull()].loc[:10000,"translations.en.title"].apply(lambda x:len(en_nlp(x)))

In [ ]:
en_desc_length = sample_df_lots["translations.en.title"].apply(lambda x:len(str(x).split(" ")))

In [ ]:
d = stats_df[stats_df["n_words"]<320000]
#sns.distplot(d["n_words"])#.save
d["n_words"]



In [ ]:
sns.distplot(en_desc_length[en_desc_length<100])

In [ ]:
sns.distplot(stats_df["n_words"])

In [ ]:
sns.distplot(en_desc_length)

In [ ]:
# length_prep_td = ungrouped_titles_descriptions.copy()
# length_prep_td["category"] = length_prep_td.index
# length_prep_td["length"] = ungrouped_titles_descriptions["translations.en.description"][ungrouped_titles_descriptions["translations.en.description"].notnull()].apply(lambda x:len(x))

In [ ]:
# sns.pointplot(x="category", y="length",data=length_prep_td)

In [ ]:
ungrouped_titles_descriptions["translations.en.description"]

### Number of categories

In [ ]:
len(df_lots["auto_category_id"].unique())

In [ ]:
#Wordcloud
title10000 = df_lots[df_lots["translations.en.title"].notnull()].loc[:100,"translations.en.title"].apply(lambda x:en_nlp(x).text).agg("sum" )
wordcloud = WordCloud().generate(title10000)

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
type(title10000)

# Feature engineering

In [ ]:
finalDF = df_lots[numeric_columns_list].sample(sample_of_rows_size)

In [ ]:
finalDF

## Simple features

In [ ]:
lots_sample_df = df_lots.loc[finalDF.index.values.tolist(),:]

### Foreign category fuzzy match

In [ ]:
df_categories[["translations.en.title","translations.de.title"]]

In [ ]:
finalSDF = pd.DataFrame(pd.Series(np.zeros(sample_of_rows_size),index = finalDF.index.values.tolist()))

In [ ]:
def fuzzMatch(st1,st21,st22):
    try:
        res = max((fuzz.ratio(st1,st21),fuzz.ratio(st1,st22)))
    except:
        res = 0
    if res > TuningParameters.fuzzy_word_threshold:
        return 1
    else:
        return 0
    
gen = df_categories[["translations.en.title","translations.de.title"]].iterrows()
for row in gen:
    finalSDF["foreign_cat_match"+str(row[0])] = lots_sample_df["foreign_category_label"].apply(fuzzMatch,args=(row[1][0],row[1][1]))

In [ ]:
finalSDF

### Foreign category in "full" description

## Advanced features

# Word vectors

In [ ]:
w2vDocmodel = {}
for k,corpus in corpuses:
    w2vDocmodel[k] += [word2vec.Word2Vec(doc.tokenized_text, size=64, window=5, min_count=1, workers=10, hs=1, negative=0)  for doc in corpus]

In [ ]:
np.avg([1,2,3])

In [ ]:
#corpuses['normal.en'][1][:20]

In [ ]:
for w2vectorDocs in w2vDocmodel["normal.en"]:
    for i,w2vectorDoc in enumerate(w2vectorDocs):
        finalSDF["w2v"+str(i)] = sample_df_lots["translations.en.full.description"].apply(lambda x:w2vectorDoc.score([x.split(" ")])[0])

In [ ]:
w2vPharsermodel = word2vec.Word2Vec(bigramPhraser[corpuses['normal.en'].text], size = 50)

# for i,w2v in enumerate(w2vDocmodel):
#     print("Document "+ str(i))
#     for crit in criteria_term_list:
#         print("Matrix = ",w2v.score(crit))
#         print("Sum = ", sum(w2v.score(crit)))

# LSA


## Vectorize TF IDF

In [ ]:
docTermMatrix10 = []
id2term = []
vectorizers = []
#docCriteriaTermIds = []
for cor in lot_corpuses.values():
    term_list = (doc.to_terms_list(ngram=1, named_entities = False, as_strings = True, filter_stops=True) for doc in cor)
    y = [doc.metadata["key"] for doc in cor]
    vectorizer = textacy.vsm.Vectorizer(weighting='tfidf', normalize=True, smooth_idf=True,min_df=2, max_df=0.95)
    docTermMatrix10 += [vectorizer.fit_transform(term_list)]
    vectorizers += [vectorizer]
    id2term += [vectorizer.id_to_term]

In [ ]:
docTermMatrix10

In [ ]:
docTermMatrix = []
id2term = []
vectorizers = []
#docCriteriaTermIds = []
for cor in corpuses.values():
    term_list = (doc.to_terms_list(ngram=1, named_entities = False, as_strings = True, filter_stops=True) for doc in cor)
    vectorizer = textacy.vsm.Vectorizer(weighting='tfidf', normalize=True, smooth_idf=True,min_df=2, max_df=0.95)
    docTermMatrix += [vectorizer.fit_transform(term_list)]
    vectorizers += [vectorizer]
    id2term += [vectorizer.id_to_term]
    
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
X = sparse_random_matrix(100, 100, density=0.01, random_state=42)
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(X) 
TruncatedSVD(algorithm='randomized', n_components=5, n_iter=7,
        random_state=42, tol=0.0)
print(svd.explained_variance_ratio_) 
[ 0.0606... 0.0584... 0.0497... 0.0434... 0.0372...]
print(svd.explained_variance_ratio_.sum()) 
0.249...

for ind,dtm in enumerate(docTermMatrix):
    df[].apply()
    term_list = (doc.to_terms_list(ngram=1, named_entities = False, as_strings = True, filter_stops=True) for doc in cor)
    rowDTM = vectorizer[ind].fit_transform(term_list)
    dotprod = np.dot(dtm,rowDTM.transpose())

In [ ]:
docTermMatrix = []
id2term = []
vectorizers = []
#docCriteriaTermIds = []
for cor in corpuses.values():
    #self, ngrams=(1, 2, 3), named_entities=True,normalize='lemma', lemmatize=None, lowercase=None,as_strings=False
    term_list = (doc.to_terms_list(ngram=1, named_entities = False, as_strings = True) for doc in cor)
    vectorizer = textacy.vsm.Vectorizer(weighting='tf',normalize='lemma', min_df=2, max_df=0.95)
    docTermMatrix += [vectorizer.fit_transform(term_list)]
    vectorizers += [vectorizer]
    id2term += [vectorizer.id_to_term]

In [ ]:
m = docTermMatrix[1]
#sdf = pd.SparseDataFrame([ pd.SparseSeries(m[i].toarray().ravel()) for i in np.arange(m.shape[0]) ]).to_csv("~/DTM_100_lemma_ngram1_ne0.csv")

np.savetxt("~/DTM_100_lemma_ngram1_ne0_de.csv", m.todense(), delimiter=',')

In [ ]:
m = docTermMatrix[0]
np.savetxt("~/DTM_100_lemma_ngram1_ne0_en_50k.csv", m[:,:50000].todense(), delimiter=',')

## Topic models

In [ ]:
# docTopicMatrix = []
# docTopicModels = []

# for dtm in docTermMatrix:
#     model = textacy.tm.TopicModel("nmf",n_topics = 5)
#     model.fit(dtm)
#     docTopicModels += [model]
#     docTopicMatrix += [model.transform(dtm)]

In [ ]:
for i,model in enumerate(docTopicModels):
    #model.tra
    #model.termite_plot(criteriaCorpDTM,criteriaId2term
    if (type(criteriaCorpDTM[i])== list and len(criteriaCorpDTM[i])==0)or(hasattr(criteriaCorpDTM,"shape") and criteriaCorpDTM[i].shape[0]==0):
        print('Cannot generate'+str(i))
        continue
    dotprod = np.dot(model.transform(criteriaCorpDTM[i]),docTopicMatrix[i].transpose())
    print(dotprod)

# Image recognition

In [ ]:
%cd ..

## Resize images

In [ ]:
path = "~/bdp_bipm/data/images/"

def saveIMG(subdir,file,path):
    im = Image.open(path)
    f, e = os.path.splitext(path)
    imResize = im.resize((224,224), Image.ANTIALIAS)
    fname= subdir.replace("/bdp_bipm/data/images/", "/bdp_bipm/data/images_224/")+"/"+ file.replace(".jpg","") + '_resized.jpg'
    os.makedirs(os.path.dirname(fname), exist_ok=True)
    #print(fname)
    imResize.save(fname, 'JPEG', quality=90)
    
def resize(p):
    pool = multiprocessing.Pool(processes=number_of_cores)
    for subdir, dirs, files in os.walk(p):
        for file in files:
            path = os.path.join(subdir, file)
            if os.path.isfile(path):
                pool.apply_async(saveIMG, args = (subdir,file,path))
                
    pool.close()
    pool.join()
                
resize(path)

In [ ]:
images_path = "~/bdp_bipm/data/images_224/"
batch_size=64

## Divide images into train and test sets

In [ ]:
image_validation_set_size = 30

In [ ]:
train_root = "~/bdp_bipm/data/images_224/train/"
valid_root = "~/bdp_bipm/data/images_224/valid/"

In [ ]:
# for item in os.listdir(train_root):
#     num_of_images = len(os.listdir(train_root+item))
#     proc_num = np.min([image_validation_set_size,int(num_of_images*0.15)])
#     if proc_num == 0:
#         continue
#     else:
#         os.makedirs(valid_root+item, exist_ok=True)
#         for i in range(proc_num):
#             newFname = random.choice(os.listdir(train_root+item))
#             src = train_root+item+"/"+ newFname
#             dest = valid_root+item+"/"+ newFname
#             shutil.copyfile(src,dest)
#             os.rename(src,train_root+item+"/clr"+ newFname)
#     #print(random.choice(os.listdir(train_root+item)))
#     #print(len(os.listdir(train_root+item)))

In [ ]:
# import utils; reload(utils)
# from utils import plots

## VGG

In [ ]:
vgg = Vgg16()

In [ ]:
#Grab a few images at a time for training and validation.
#NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(images_path+'train')
val_batches = vgg.get_batches(images_path+'valid')

In [ ]:
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_steps=1000, nb_epoch=1)

In [ ]:
vgg.finetune(batches)
vgg.fit(batches, nb_steps=5000, nb_epoch=3)

In [ ]:
imgs,labels = next(batches)

In [ ]:
plots(imgs[:3], titles=labels[:3])

In [ ]:
vgg.predict(imgs[:3], True)

In [ ]:
batches.num_class

In [ ]:
vgg.finetune(batches)

In [ ]:
vgg.fit(batches, val_batches, nb_steps=3000, nb_epoch=1)

In [ ]:
imgs,labels = next(val_batches)

In [ ]:
test_array = vgg.test(images_path+'valid')

In [ ]:
test_array

# Prediction

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(converted_features, converted_target, test_size = 0.2, random_state=21)

imp = Imputer(missing_values=np.NaN, strategy='median', axis=0)
converted_df = pd.DataFrame(imp.fit_transform(converted_df), columns=converted_df.columns)

scaler = StandardScaler()

#pipeline = Pipeline([
#    ('vect', CountVectorizer()),
#    ('tfidf', TfidfTransformer()),
#    ('clf', SGDClassifier()),
#])
converted_df = pd.DataFrame(scaler.fit_transform(converted_df), columns=converted_df.columns)

## Naive Bayes

In [ ]:
#Naive Bayes and foundation for CrossValidation
imputer = preprocessing.Imputer(np.nan)

scaler = preprocessing.StandardScaler()

clf = GaussianNB()

pipeline = Pipeline([("impute",imputer),('scale', scaler), ('nb', clf)])

def nonNumberToNan(x):
    try:
        int(x)
        return x
    except:
        return np.nan

X = df_lots[numeric_columns_list].apply(lambda row:row.apply(nonNumberToNan)).values
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_lots[target_column].values)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(docTermMatrix, y, test_size=0.23, random_state=42)
#pipeline.fit(X_train, y_train)
#pipeline.score(X_test,y_test)

In [ ]:
X_train = docTermMatrix[:80000]
y_train = y[:80000]
X_valid =  docTermMatrix[80000:100000]
y_valid = y[80000:100000]

## Random Forest H2O

In [ ]:
import h2o

h2o.init(max_mem_size = "2G")             #specify max number of bytes. uses all cores by default.
h2o.remove_all()

from h2o.estimators.random_forest import H2ORandomForestEstimator

covtype_df = h2o.import_file(os.path.realpath("../data/covtype.full.csv"))


# We import the full covertype dataset (581k rows, 13 columns, 10 numerical, 3 categorical) and then split the data 3 ways:  
#   
# 60% for training  
# 20% for validation (hyper parameter tuning)  
# 20% for final testing  
# 
#  We will train a data set on one set and use the others to test the validity of the model by ensuring that it can predict accurately on data the model has not been shown.  
#  
#  The second set will be used for validation most of the time.  
#  
#  The third set will be withheld until the end, to ensure that our validation accuracy is consistent with data we have never seen during the iterative process. 

# In[ ]:

#split the data as described above
train, valid, test = covtype_df.split_frame([0.6, 0.2], seed=1234)

#Prepare predictors and response columns
covtype_X = covtype_df.col_names[:-1]     #last column is Cover_Type, our desired response variable 
covtype_y = covtype_df.col_names[-1]    



rf_v1 = H2ORandomForestEstimator(
    model_id="rf_covType_v1",
    ntrees=200,
    stopping_rounds=2,
    score_each_iteration=True,
    seed=1000000)



rf_v1.train(covtype_X, covtype_y, training_frame=train, validation_frame=valid)


rf_v1.score_history()


# Here we can see the hit ratio table.

# In[ ]:

rf_v1.hit_ratio_table(valid=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=20)
clf.fit(docTermMatrix[0], y) 

In [ ]:
param_grid = {"max_depth": [3, 7, None],
              "bootstrap": [True, False],
              "max_features": [3,6,9],
              "min_samples_split": [2,6,8],
              "min_samples_leaf": [2,6,8]}

grid_search = GridSearchCV(clf, param_grid=param_grid, cv = 5)

grid_search.fit(docTermMatrix, y_train)

rf_pred = grid_search.predict_proba(X_test) #needed for the ROC curve
rf_pred2 = grid_search.predict(X_test) #needed for the confusion matrix

print("rf"+str(rf_pred[:,0]))
print(grid_search.score(X_test, y_test))
print("LogLoss = "+str(log_loss(y_test, rf_pred)))
print(confusion_matrix(y_test, rf_pred2))
print(classification_report(y_test, rf_pred2))

rf_pred_prob = grid_search.predict_proba(X_test)[:,1]
rf_fpr, rf_tpr, thresholds = roc_curve(y_test, rf_pred_prob)

plt.plot([0, 1], [0, 1], 'k--')
plt.plot(rf_fpr, rf_tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

## SVM

In [ ]:
docTermMatrix 

In [ ]:
X_test = svd.fit_transform(docTermMatrix10[0]) 

In [ ]:
from sklearn import svm
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
X = svd.fit_transform(docTermMatrix[0]) 
clf = svm.SVC(decision_function_shape='ovo')

In [ ]:
clf.fit(X, y) 

In [ ]:
clf.score(X_test[:5000],y[:5000])

## XGBoost

In [ ]:
# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_test, label=y_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

#xgb.fit(X_train, y_train)

d_test = xgb.DMatrix(X_test)
p_test = bst.predict(d_test)

#xgb_pred = xgb.predict(X_test)

print("LogLoss = " + str(log_loss(y_test, p_test)))